<a href="https://colab.research.google.com/github/Thibooooo/Optimisation-Course/blob/main/Optimization_E6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Given the following LP problem (the primal problem):

Maximize:

$x1+4⋅x2+2⋅x3$

Subject to:

$5⋅x1+2⋅x2+2⋅x3≤145$

$4⋅x1+8⋅x2−8⋅x3≤260$

$x1+x2+4⋅x3≤190$

1. We convert the LP problem to a dual LP problem:

Minimize:
    
$145⋅y1+260⋅y2+190⋅y3$

Subject to:
   
$5⋅y1+4⋅y2+y3>=1$.

$2⋅y1+8⋅y2+y3>=4$.

$2⋅y1-8⋅y2+4⋅y3>=2$.  



2. To verify $Q=(x_1,x_2,x_3)=(0,52.5,20)$ is a feasible solution, we need to check if this solution meets all the constraints:

Constraint 1: $5*0+2*52.5+2*20<=145$.  
This is true as $0+105+40=145<=145$.

Constraint 2: $4⋅0+8⋅52.5−8⋅2<=260$.  
This is also true, as $0+420−160=260≤260$.

Constraint 3: $0+52.5+4⋅20<=190$.  
This is also true, as $52.5+80=132.5<=190$




\begin{document}

Problem Primal
\[
\begin{array}{|c|c c c|c|}
\hline
\text{Maximize} & x_1 & x_2 & x_3 & \\
\hline
\text{Subject to} & 5x_1 & + 2x_2 & + 2x_3 & \leq 145 \\
& 4x_1 & + 8x_2 & - 8x_3 & \leq 260 \\
& x_1 & + x_2 & + 4x_3 & \leq 190 \\
\hline
\end{array}
\]

Problem Dual
\[
\begin{array}{|c|c c c|c|}
\hline
\text{Minimize} & y_1 & y_2 & y_3 & \\
\hline
\text{Subject to} & 5y_1 & + 4y_2 & + y_3 & \geq 1 \\
& 2y_1 & + 8y_2 & + y_3 & \geq 4 \\
& 2y_1 & - 8y_2 & + 4y_3 & \geq 2 \\
\hline
\end{array}
\]

\end{document}


3. We use complimentary slackness to determine a candidate solution to the dual

In [1]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.4.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.

In [4]:
from ortools.linear_solver import pywraplp

def solve_lp(c, A, b):
    solver = pywraplp.Solver.CreateSolver('GLOP')

    n = len(c)
    x = [solver.NumVar(0, solver.infinity(), 'x{}'.format(i)) for i in range(n)]

    for i in range(len(A)):
        constraint_expr = solver.RowConstraint(-solver.infinity(), b[i], '')
        for j in range(len(A[i])):
            constraint_expr.SetCoefficient(x[j], A[i][j])

    objective = solver.Objective()
    for j in range(n):
        objective.SetCoefficient(x[j], c[j])
    objective.SetMaximization()

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        for i in range(n):
            print('x{} = {}'.format(i, x[i].solution_value()))
    else:
        print('The problem does not have an optimal solution.')



In [5]:
#Ex
c = [1, 4, 2]
A = [[5, 2, 2], [4, 8, -8], [1, 1, 4]]
b = [145, 260, 190]

solve_lp(c, A, b)

Solution:
Objective value = 249.99999999999994
x0 = 0.0
x1 = 52.49999999999999
x2 = 19.999999999999993


In [3]:
def print_primal_dual_tables(c, A, b, tableau_intermediaire=False):
    n = len(c)
    m = len(A)
    tableau_primal = []
    tableau_dual = []

    # Problème Primal
    primal_str = "Problem Primal:\n"
    primal_str += "| Maximize | "
    for i in range(n):
        primal_str += f"{c[i]}*x{i+1} "
    primal_str += "|\n"

    primal_str += "| Subject to | "
    for i in range(m):
        primal_str += "| "
        for j in range(n):
            primal_str += f"{A[i][j]}*x{j+1} "
        primal_str += f"<= {b[i]} |\n"
    primal_str += "\n"
    tableau_primal.append(primal_str)


    dual_str = "Problem Dual:\n"
    dual_str += "| Minimize | "
    for i in range(m):
        dual_str += f"{b[i]}*y{i+1} "
    dual_str += "|\n"

    for j in range(n):
        dual_str += "| Subject to | "
        for i in range(m):
            dual_str += f"{A[i][j]}*y{i+1} "
        dual_str += ">= "
        if c[j] == 1:
            dual_str += "1 |\n"
        else:
            dual_str += f"{c[j]} |\n"
    dual_str += "\n"
    tableau_dual.append(dual_str)


    if tableau_intermediaire:
        for i in range(len(tableau_primal)):
            print("Tableau intermédiaire", i+1, ":\n")
            print(tableau_primal[i])
            print(tableau_dual[i])

def solve_lp(c, A, b, tableau_intermediaire=False):
    solver = pywraplp.Solver.CreateSolver('GLOP')

    n = len(c)
    x = [solver.NumVar(0, solver.infinity(), 'x{}'.format(i)) for i in range(n)]

    for i in range(len(A)):
        constraint_expr = solver.RowConstraint(-solver.infinity(), b[i], '')
        for j in range(len(A[i])):
            constraint_expr.SetCoefficient(x[j], A[i][j])

    objective = solver.Objective()
    for j in range(n):
        objective.SetCoefficient(x[j], c[j])
    objective.SetMaximization()

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        for i in range(n):
            print('x{} = {}'.format(i, x[i].solution_value()))
    else:
        print('The problem does not have an optimal solution.')

    if tableau_intermediaire:
        print_primal_dual_tables(c, A, b, tableau_intermediaire=True)


c = [1, 4, 2]
A = [[5, 2, 2], [4, 8, -8], [1, 1, 4]]
b = [145, 260, 190]

solve_lp(c, A, b, tableau_intermediaire=True)

#I've tried to show a array for each


Solution:
Objective value = 249.99999999999994
x0 = 0.0
x1 = 52.49999999999999
x2 = 19.999999999999993
Tableau intermédiaire 1 :

Problème Primal:
| Maximize | 1*x1 4*x2 2*x3 |
| Subject to | | 5*x1 2*x2 2*x3 <= 145 |
| 4*x1 8*x2 -8*x3 <= 260 |
| 1*x1 1*x2 4*x3 <= 190 |


Problème Dual:
| Minimize | 145*y1 260*y2 190*y3 |
| Subject to | 5*y1 4*y2 1*y3 >= 1 |
| Subject to | 2*y1 8*y2 1*y3 >= 4 |
| Subject to | 2*y1 -8*y2 4*y3 >= 2 |




4. The objective function's value is approximately 250.  
The optimal values for the decision variables are:  
x1=0.  
x2=52,5.  
x3=20.  

This matches the objective value provided earlier indicating that the solution is optimal. Therefore, the solution $Q=(x_1,x_2,x_3)=(0,52.5,20)$ satisfies both feasibility and optimality, suggesting that it is indeed the optimal solution to the primal problem.


